# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import re

# Avoid too many decimal places on Pandas output
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Auxiliary functions

In [14]:
def print_correlations(df):
    rho = df.corr()
    pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
    print(rho.round(2).astype(str) + p)
    
def plot_correlations(df): 
    corr = df.corr()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
    fig.colorbar(cax)
    ticks = np.arange(0, len(df.columns), 1)
    ax.set_xticks(ticks)
    plt.xticks(rotation=90)
    ax.set_yticks(ticks)
    ax.set_xticklabels(df.columns)
    ax.set_yticklabels(df.columns)
    plt.show()

# Load data

In [18]:
evaluation_path = './experiments/evaluation_error_stable.csv'

# Remove special characters output if they are present
string = open(evaluation_path).read()
new_str = re.sub('\[m', '', string)
open(evaluation_path, 'w').write(new_str)

28072

In [19]:
df = pd.read_csv(evaluation_path)

# Remove columns unnecessary for the analysis
df2 = df.drop(columns=['model', 'smooth_v', 'inner', 'run'])
df2.head()

,mean_error,last_error,n-links,alpha,inner_epsilon,rehu
0,13223.76774,16382.97897,8,0.00010,0.20000,0.00050
1,43904.14910,83817.34390,8,0.00010,0.01000,0.00050
2,6863.59554,10042.53480,8,0.00010,0.00500,0.00050
3,9318.50523,21328.84341,8,0.00010,0.00100,0.00050
4,2671.31691,3360.66257,8,0.00010,0.00050,0.00050


# Evaluation analysis

## Mean error by hyperparameter configuration

In [20]:
df_mean = df2.groupby(by=['n-links', 'alpha', 'inner_epsilon', 'rehu']).mean().sort_values(by=['n-links', 'mean_error'])
df_mean

mean_error      last_error
n-links alpha   inner_epsilon rehu                                  
2       0.00005 0.20000       0.00050     5901.24338      9740.11789
        0.00010 0.00050       0.00050     6115.86219      6537.31125
                0.00010       0.00050     8116.96515     15687.91266
        0.00005 0.00050       0.00050     8876.59616     11933.58406
        0.00010 0.20000       0.00050     9286.64429     13464.37329
                0.10000       0.00500     9963.84236     15569.64599
                0.05000       0.00500    11517.57052     16814.07624
                0.01000       0.00050    13113.35350     32439.25008
        0.00005 0.00500       0.00050    15924.70978     23698.96556
                0.01000       0.00050    24856.59456     33220.12657
        0.00010 0.00100       0.00050    25526.02316     39486.69153
                0.00500       0.00050    28446.90664     40967.10755
                0.01000       0.00500    32630.54588     49842.06135
        0.00005 0.00100       0.00050    35358.09668     66018.88808
                0.00010       0.00050    52416.78325     87577.31029
8       0.00005 0.20000       0.00050     5902.54047      8445.24912
        0.00010 0.20000       0.00050     8088.16350     11603.90157
        0.00005 0.01000       0.00050    11689.66608     26212.20233
                0.00100       0.00050    12996.22103     61415.57856
        0.00010 0.01000       0.00050    14320.92214     31306.36264
        0.00005 0.00500       0.00050    20662.60996     66033.49121
        0.00010 0.00500       0.00050    29573.60950    127609.23641
                0.10000       0.00500    35378.90579     51196.50832
                0.05000       0.00500    42888.96656     60514.20088
                0.01000       0.00500   214454.04534    757175.67800
                0.00100       0.00050   315114.56173   2890797.32557
        0.00005 0.00010       0.00050   432107.63205   6560031.38989
                0.00050       0.00050  2672059.72576  16844845.18626
        0.00010 0.00050       0.00050  7298058.82296  41740475.93165
                0.00010       0.00050 33086092.11623 345484090.68656

In [23]:
for n in df2['n-links'].unique():
    df3 = df2[df2['n-links'] == n]
    print("N-links:", n, "Size", len(df3))
    print_correlations(df3)
    print_correlations(df3)

              mean_error last_error n-links    alpha inner_epsilon     rehu
mean_error        1.0***    0.99***     nan     0.06         -0.05    -0.04
last_error       0.99***     1.0***     nan     0.06         -0.05    -0.04
n-links              nan        nan  nan***      nan           nan      nan
alpha               0.06       0.06     nan   1.0***          0.04  0.41***
inner_epsilon      -0.05      -0.05     nan     0.04        1.0***      0.1
rehu               -0.04      -0.04     nan  0.41***           0.1   1.0***
N-links: 8 Size 225
              mean_error last_error n-links    alpha inner_epsilon     rehu
mean_error        1.0***    0.99***     nan     0.06         -0.05    -0.04
last_error       0.99***     1.0***     nan     0.06         -0.05    -0.04
n-links              nan        nan  nan***      nan           nan      nan
alpha               0.06       0.06     nan   1.0***          0.04  0.41***
inner_epsilon      -0.05      -0.05     nan     0.04        1.0***  

/home/snop/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/snop/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/snop/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/snop/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [81]:
df_mean = df2.groupby(by=['n-links', 'alpha', 'inner_epsilon', 'rehu']).mean().sort_values(by=['n-links', 'mean_error'])
df_mean

mean_error     last_error
n-links alpha   inner_epsilon rehu                                 
2       0.00010 0.00500       0.00050    17755.03218    29907.97635
                0.00100       0.00050    18733.91153    28594.62631
        0.00100 0.01000       0.00100    21265.70132    35702.30002
                              0.00500    24474.06733    40222.31988
                0.00100       0.00100    29121.99276    52071.89885
                0.01000       0.00050    31322.47854    58288.16258
                0.00500       0.00100    32068.10100    51050.33401
                              0.00050    32549.30920    55569.09605
                0.00100       0.00500    33023.24943    57030.63082
                              0.00050    33817.06902    50778.32782
        0.00010 0.00500       0.00500    37457.53639    74554.18915
                0.00100       0.00100    37947.78453    63362.79498
                0.00500       0.00100    43919.96579    76819.42175
                0.01000       0.00500    50150.71779    81254.67094
                              0.00050    50776.46473    81997.24687
        0.00100 0.00500       0.00500    58413.08689   111227.35984
        0.00010 0.01000       0.00100    65315.19145   146865.16116
                0.00100       0.00500   141148.44781   408987.32494
5       0.00100 0.01000       0.00500    41100.47770    81117.56817
                              0.00050    42741.98824    94090.56394
                              0.00100    66014.44257   156877.94507
        0.00010 0.01000       0.00100    91896.39659   217153.51656
                              0.00500   104048.13746   215213.08279
        0.00100 0.00500       0.00500   185305.98323   477108.10948
        0.00010 0.01000       0.00050   254458.27641   612878.52625
                0.00500       0.00100   335076.04494   824847.91443
                              0.00500   335950.12773  1067398.52861
                              0.00050   479557.34101  1366449.74883
        0.00100 0.00500       0.00100   870883.38794  2198315.72915
                              0.00050   957882.39225  2248312.39473
        0.00010 0.00100       0.00100  6042418.44160 17220543.28755
        0.00100 0.00100       0.00100  8195146.29052 19243930.04200
                              0.00500  8320640.97215 22138154.81195
        0.00010 0.00100       0.00050 13342993.24064 39011688.73631
                              0.00500 14817552.13528 38992951.52100
        0.00100 0.00100       0.00050 20394525.24265 48283367.22375
8       0.00010 0.00500       0.00050    18632.07556    55067.18821
        0.00100 0.01000       0.00050    23586.00707    60412.06911
        0.00010 0.01000       0.00050    27077.45122    65559.78015
                0.00100       0.00050    78367.92222   503642.10267
        0.00100 0.01000       0.00100   148631.63192   431716.84569
        0.00010 0.01000       0.00500   169274.72058   437833.00920
        0.00100 0.00500       0.00050   226142.87296   619296.41318
        0.00010 0.01000       0.00100   451543.33777  1075374.50172
        0.00100 0.00500       0.00500  1178444.77462  2939081.73137
        0.00010 0.00500       0.00500  1407909.41885  3264513.92656
        0.00100 0.00500       0.00100  1651823.85018  4463203.89766
                0.01000       0.00500  2150648.36868  5141059.96953
        0.00010 0.00500       0.00100  2440953.30593  5676424.95707
        0.00100 0.00100       0.00050  3267394.55311 13714442.05333
        0.00010 0.00100       0.00500  4873243.03760 14211377.82848
        0.00100 0.00100       0.00100  6257916.26436 14913459.19087
                              0.00500  8713887.16141 19324463.36685
        0.00010 0.00100       0.00100  9736526.00877 41192724.78825

In [82]:
# Look at specific hyperparameter combinations
df_mean.query('alpha==0.00010 & inner_epsilon==0.00100 & rehu==0.00100')

,,,,mean_error,last_error
n-links,alpha,inner_epsilon,rehu,,
2,0.00010,0.00100,0.00100,37947.78453,63362.79498
5,0.00010,0.00100,0.00100,6042418.44160,17220543.28755
8,0.00010,0.00100,0.00100,9736526.00877,41192724.78825


In [83]:
df_min = df2.groupby(by=['n-links', 'alpha', 'inner_epsilon', 'rehu']).min().sort_values(by=['mean_error'])
df_min

mean_error  last_error
n-links alpha   inner_epsilon rehu                           
2       0.00100 0.00500       0.00100   186.07648    49.58420
        0.00010 0.00100       0.00100   209.28250    98.22165
                              0.00050   217.19174   116.40142
        0.00100 0.00500       0.00050   217.72774    80.98180
                0.00100       0.00050   221.42107   121.31620
                              0.00100   224.88387   101.30634
        0.00010 0.00500       0.00050   244.16226   129.58119
        0.00100 0.01000       0.00050   272.33922   141.16064
                              0.00100   282.94822   131.68829
        0.00010 0.01000       0.00100   410.54666   403.18702
        0.00100 0.00500       0.00500   549.37160   542.68587
        0.00010 0.00500       0.00100  1121.86375  1350.70020
5       0.00010 0.01000       0.00100  1168.51029   942.00263
2       0.00010 0.01000       0.00050  1181.14941  1329.60892
        0.00100 0.01000       0.00500  1234.83182  1241.74094
5       0.00010 0.00100       0.00050  1381.16247  1010.98482
8       0.00010 0.01000       0.00050  1413.57246   964.38045
2       0.00100 0.00100       0.00500  1512.03371  1784.78384
8       0.00010 0.00100       0.00100  1571.16030  1177.13135
                              0.00050  1603.87510  1581.38093
5       0.00010 0.00500       0.00500  1670.38741  1290.52226
        0.00100 0.01000       0.00100  1870.91071  1479.68399
2       0.00010 0.01000       0.00500  2019.43070  2163.44086
8       0.00010 0.00500       0.00050  2020.60577  1791.21774
        0.00100 0.00100       0.00050  2055.32561  2506.99694
                0.00500       0.00050  2166.40877  2454.51949
5       0.00100 0.00100       0.00050  2191.63715  1682.76286
                0.01000       0.00500  2266.57885  1873.89403
8       0.00100 0.00500       0.00100  2329.09672  2456.16254
5       0.00100 0.01000       0.00050  2422.38641  2649.82541
8       0.00100 0.01000       0.00050  2431.65900  2982.33576
                              0.00100  2629.14212  3228.14157
5       0.00010 0.01000       0.00050  2840.51296  2665.35854
8       0.00010 0.00500       0.00100  2877.69875  4759.53649
5       0.00100 0.00500       0.00100  2935.00420  2138.81993
2       0.00010 0.00500       0.00500  2936.93343  1774.45230
8       0.00100 0.00100       0.00100  2999.31816  3666.85402
5       0.00010 0.00500       0.00050  3050.24952  3555.51439
                0.00100       0.00500  3360.55529  2140.90481
        0.00100 0.00100       0.00500  3482.47570  4135.83541
8       0.00100 0.00500       0.00500  3588.63235  3726.44821
5       0.00010 0.00500       0.00100  3706.91801  3444.53759
        0.00100 0.00500       0.00050  3910.49539  3744.96427
                              0.00500  4316.99509  4039.30100
8       0.00010 0.01000       0.00100  4364.21226  6976.12698
5       0.00100 0.00100       0.00100  4375.10420  4241.87553
        0.00010 0.00100       0.00100  6021.66549  6270.01498
                0.01000       0.00500  6549.68154  5830.63341
8       0.00100 0.00100       0.00500  9215.47726 41202.83070
        0.00010 0.00100       0.00500  9227.74695 15895.33639
2       0.00010 0.00100       0.00500  9848.99853 12098.52882
8       0.00010 0.00500       0.00500 13962.78303 22681.15176
                0.01000       0.00500 28554.25828 42146.31285
        0.00100 0.01000       0.00500 35196.60587 65471.81879